In [2]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pylab as plt
import numpy as np

In [3]:
img_height,img_width=(224,224)
batch_size=12
train_data_dir=r"dataset\train"
test_data_dir=r"dataset\train"
print("====================================================")

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.4)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='validation')



test_generator = train_datagen.flow_from_directory(test_data_dir,
                                                target_size=(img_height,img_width),
                                                batch_size=1,
                                                class_mode='categorical',
                                                subset='validation')
x,y=test_generator.next()
x.shape

Found 606 images belonging to 2 classes.
Found 606 images belonging to 2 classes.


(1, 224, 224, 3)

### **Rest net**

In [10]:
base_model=ResNet50(include_top=False,weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(train_generator.num_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=predictions)

for layer in base_model.layers:
    layer.trainable=False

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,
          epochs=5,
        validation_data=test_generator)

Epoch 1/5
51/51 [==============================] - 158s 3s/step - loss: 1.2217 - accuracy: 0.7459 - val_loss: 0.1901 - val_accuracy: 0.9505
Epoch 2/5
51/51 [==============================] - 157s 3s/step - loss: 0.1421 - accuracy: 0.9488 - val_loss: 0.0696 - val_accuracy: 0.9818
Epoch 3/5
51/51 [==============================] - 158s 3s/step - loss: 0.0701 - accuracy: 0.9851 - val_loss: 0.0387 - val_accuracy: 0.9917
Epoch 4/5
51/51 [==============================] - 160s 3s/step - loss: 0.0267 - accuracy: 0.9967 - val_loss: 0.0321 - val_accuracy: 0.9917
Epoch 5/5
51/51 [==============================] - 157s 3s/step - loss: 0.0199 - accuracy: 0.9983 - val_loss: 0.0125 - val_accuracy: 0.9967


In [11]:
# Save the trained model in .h5 format
model.save(r"resnet_model.h5")

C:\Users\0819\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
vgg_acc=history.history['val_accuracy'][-1]
print(vgg_acc)

0.9966996908187866


### **MobileNet**

In [13]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the MobileNet model
base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

# Add custom layers on top of MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of MobileNet to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=test_generator)

# Save the trained model in .h5 format
model.save(r"mobilenet_model.h5")


17225924/17225924 [==============================] - 3s 0us/step
Epoch 1/5
51/51 [==============================] - 100s 2s/step - loss: 0.6818 - accuracy: 0.7690 - val_loss: 0.2468 - val_accuracy: 0.9092
Epoch 2/5
51/51 [==============================] - 94s 2s/step - loss: 0.1938 - accuracy: 0.9307 - val_loss: 0.1016 - val_accuracy: 0.9785
Epoch 3/5
51/51 [==============================] - 92s 2s/step - loss: 0.1258 - accuracy: 0.9604 - val_loss: 0.0934 - val_accuracy: 0.9719
Epoch 4/5
51/51 [==============================] - 92s 2s/step - loss: 0.1752 - accuracy: 0.9208 - val_loss: 0.3463 - val_accuracy: 0.8399
Epoch 5/5
51/51 [==============================] - 89s 2s/step - loss: 0.0996 - accuracy: 0.9670 - val_loss: 0.0224 - val_accuracy: 0.9983


In [14]:
mobilenet_acc=history.history['val_accuracy'][-1]
print(mobilenet_acc)

0.9983498454093933


In [ ]:
from sklearn.metrics import classification_report #classification report gives precision, recall, and F1-score for each class.
from tensorflow.keras.models import load_model
import numpy as np
# Load the trained MobileNet model
model = load_model(r"mobilenet_model.h5")

# Generate predictions on the test set
predictions = model.predict(test_generator)

# Get the predicted class indices
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes from the test generator
true_classes = test_generator.classes

# Get the class labels
class_labels = list(test_generator.class_indices.keys())

# Generate the classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the classification report
print(report)

606/606 [==============================] - 52s 84ms/step
              precision    recall  f1-score   support

    negative       0.53      0.54      0.53       320
    positive       0.48      0.47      0.47       286

    accuracy                           0.51       606
   macro avg       0.50      0.50      0.50       606
weighted avg       0.51      0.51      0.51       606



### **DenseNet**

In [18]:
from tensorflow.keras.applications import DenseNet121
# Load the DenseNet121 base model (without the top classification layers)
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

# Add custom layers on top of the DenseNet model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Fully connected layer with 1024 units
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Final output layer

# Create the final model
dense = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with Adam optimizer and categorical crossentropy loss
dense.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the generators
history = dense.fit(train_generator,
                    epochs=5,
                    validation_data=test_generator)


Epoch 1/5
51/51 [==============================] - 173s 3s/step - loss: 3.9941 - accuracy: 0.5875 - val_loss: 1.6643 - val_accuracy: 0.5941
Epoch 2/5
51/51 [==============================] - 163s 3s/step - loss: 0.9787 - accuracy: 0.7492 - val_loss: 0.8955 - val_accuracy: 0.7822
Epoch 3/5
51/51 [==============================] - 161s 3s/step - loss: 0.7373 - accuracy: 0.7805 - val_loss: 0.4779 - val_accuracy: 0.8201
Epoch 4/5
51/51 [==============================] - 153s 3s/step - loss: 0.6949 - accuracy: 0.7921 - val_loss: 0.3691 - val_accuracy: 0.8564
Epoch 5/5
51/51 [==============================] - 155s 3s/step - loss: 0.2432 - accuracy: 0.8977 - val_loss: 0.2406 - val_accuracy: 0.9059


In [19]:
# Save the trained model in .h5 format
dense.save(r"DenseNet_model.h5")

C:\Users\0819\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
densenet_acc=history.history['val_accuracy'][-1]
print(densenet_acc)

0.905940592288971


In [6]:
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
import numpy as np
# Load the trained MobileNet model
model = load_model(r"DenseNet_model.h5")

# Generate predictions on the test set
predictions = model.predict(test_generator)

# Get the predicted class indices
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes from the test generator
true_classes = test_generator.classes

# Get the class labels
class_labels = list(test_generator.class_indices.keys())

# Generate the classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the classification report
print(report)

606/606 [==============================] - 73s 117ms/step
              precision    recall  f1-score   support

    negative       0.53      0.63      0.58       320
    positive       0.48      0.37      0.42       286

    accuracy                           0.51       606
   macro avg       0.50      0.50      0.50       606
weighted avg       0.50      0.51      0.50       606



### **CNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Building the CNN model
cnn_model = Sequential()

# First Convolutional Layer
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolutional Layer
cnn_model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
cnn_model.add(Flatten())

# Fully Connected Layer using neurons 
cnn_model.add(Dense(1024, activation='relu'))
cnn_model.add(Dropout(0.5))

# Output Layer (Softmax for multi-class classification)
cnn_model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = cnn_model.fit(train_generator,
                        epochs=5,
                        validation_data=test_generator)


Epoch 1/5
51/51 [==============================] - 165s 3s/step - loss: 57.6119 - accuracy: 0.4917 - val_loss: 0.6927 - val_accuracy: 0.5281
Epoch 2/5
51/51 [==============================] - 163s 3s/step - loss: 0.6926 - accuracy: 0.5281 - val_loss: 0.6922 - val_accuracy: 0.5281
Epoch 3/5
51/51 [==============================] - 157s 3s/step - loss: 0.6923 - accuracy: 0.5281 - val_loss: 0.6920 - val_accuracy: 0.5281
Epoch 4/5
51/51 [==============================] - 151s 3s/step - loss: 0.6923 - accuracy: 0.5281 - val_loss: 0.6917 - val_accuracy: 0.5281
Epoch 5/5
51/51 [==============================] - 152s 3s/step - loss: 0.6918 - accuracy: 0.5281 - val_loss: 0.6917 - val_accuracy: 0.5281


In [23]:
# Save the trained model in .h5 format
cnn_model.save(r"cnn_model.h5")

C:\Users\0819\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
cnn_acc=history.history['val_accuracy'][-1]
print(cnn_acc)

0.528052806854248
